# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)


In [ ]:
import numpy as np
import json
from datasets import load_dataset
from random import shuffle
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

In [ ]:
# Ako imamo vise ponudenih odgovora od 4, da postupak bude automatski u svim funkcijama
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'][3::-1]
letters, len(letters)

## Metrike

Preuzeto direktno s njihovog repoa

In [ ]:
import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

def getResultdata(result_data):
    choice_to_index = {letters[i]: i for i in range(len(letters))}
    choice_to_index[None] = len(letters)

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1
                
    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)
    
    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    word_data_list = list(word_play.values())
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)
    
    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')   
    final_result['sentence'] = getMetric(reverse_data_list)  
    
    
    all_data = word_data_list + reverse_data_list
    print('#########All data##########') 
    final_result['all'] = getMetric(all_data) 
    
    return final_result

### Spremanje datoteka

Automatski dodaje +1 na naziv ako vec postoji takva datoteka

Uzima format model_name + name + redni broj za datoteku

Unutra pise prvi testni primjer (example) i rezultate

In [ ]:
def save(name=None, few_shot=False, example=format):
    try:
        os.mkdir('results')
    except:
        pass
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    while True:
        file_path = f'results/{model_name}{inserted_name}_{i}.txt'
        try:
            with open(file_path, 'r'):
                i += 1
        except:
            with open(file_path, 'w') as file:
                file.write(model_name+'\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if not isinstance(example, str):
                    example = json.dumps(example)
                file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4))
                return

### Load the model

Login kako bi se mogli ucitati napredniji modeli

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
# model_path = 'microsoft/phi-2'
# model_path = 'microsoft/Phi-3-mini-128k-instruct'
model_path = 'microsoft/Phi-3-mini-4k-instruct'

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

if model_path.find('flan') >= 0:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path, trust_remote_code=True, device_map = 'cuda')
else:
    model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, device_map = 'cuda')

### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`, ako na Kaggleu, onda napraviti svoj dataset te ga dodati pod input

In [ ]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
data_path = '/kaggle/input/brainteaser/data'
sentence_data_path = f"{data_path}/SP-train.npy"
wordplay_data_list = f"{data_path}/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

In [ ]:
newline = '\n'
format = f"""Question: {"{}"}
Choice:
{''.join('(' + x + ') {}' + newline for x in letters)}
Answer:("""
print(format)

In [ ]:
def get_single_demo(sample):
    sample_demo = format.format(
        sample['question'], *sample['choice_list'])
    return sample_demo

good_responses = [f'{x})' for x in letters]
print("good_responses", good_responses)

## Generiranje dataseta

### Pomocne funkcije

In [ ]:
def set_predictions(data_list):
    """Uzima response i parsira ga tako da pod predict stavi slovo koje je napisano"""
    for index,item in enumerate(data_list):
        item['predict'] = None
        for x in letters:
            if (f'{x})') in item['response']:
                item['predict'] = x

        if item['predict'] is None:
            print(index)

def custom_stopping_criteria(input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs) -> bool:
    """
    Funkcija kako bi se dinamicki prepoznalo kad treba prestati generirati tekst:
    ne nakon fiksnog broja tokena, vec kad model napise rjesenje. Najcesce
    ce to ipak biti medu prva dva tokena
    """
    decoded = tokenizer.decode(input_ids[0][-3:])
    for good_response in good_responses:
        if good_response in decoded:
            return True
    return False

stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [ ]:
def generate(samples, tokens=10, all_tokens=True, few_shot=False):
    for sample in tqdm(samples):
        if few_shot:
            text = demonstration + get_single_demo(sample)
        else:
            text = get_single_demo(sample)
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(device)
        original_tokens = len(inputs[0])
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            do_sample = False,
            max_new_tokens=tokens,
            stopping_criteria=stopping_criteria
        )
        outputs = outputs[0][0 if all_tokens else original_tokens:]
        sample['response'] = tokenizer.decode(outputs)
    set_predictions(samples)

In [ ]:
def create_with_question(i):
    """Generira sample tako da zamijeni drugi distraction s pitanjem"""
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    choice_list[current['choice_order'].index(2)] = current['question'][:-1]
    new['choice_list'] = choice_list
    return new

def test_with_question(generated_data_list):
    """Testira radi li generiranje ispravno"""
    for i, sample in enumerate(generated_data_list):
        if i in [227, 755, 714, 715, 716]: continue
        assert sample['distractor2'] not in sample['choice_list'], i
        assert sample['question'][:-1] in sample['choice_list'], i
        assert str(sample['answer']) in sample['choice_list'], i
        assert str(sample['distractor1']) in sample['choice_list'], i

def count_with_question(generated_data_list):
    """Broji koliko puta je model predvidio da je pitanje odgovor na pitanje"""
    total = 0
    for sample in generated_data_list:
        if sample['choice_list'][letters.index(sample['predict'])] == sample['question'][:-1]:
            total += 1
    return total, len(generated_data_list)

def create_with_ordered_correct_answer(i, target_position):
    """Generira sample tako da je odgovor uvijek na target position"""
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    correct_answer = choice_list[current["label"]]
    choice_list.remove(correct_answer)
    choice_list.insert(target_position,correct_answer)
    new['choice_list'] = choice_list
    new['label'] = target_position
    return new

def test_with_ordered_correct_answer(generated_data_list):
    """Testira radi li generiranje ispravno"""
    for original, new in zip(test_data_list, generated_data_list):
        assert set(original['choice_list']) == set(new['choice_list']), str(original) + '\n'+ str(new)
        assert new['choice_list'][new['label']] == original['choice_list'][original['label']]

def create_with_k_answers(i, k):
    """Generira sample tako da doda kao dodatne odgovore distractore iz drugih pitanja"""
    current = test_data_list[i]
    choice_list = [*current['choice_list']]
    choice_order = [*current['choice_order']]
    while len(choice_list) < k-1:
        i -= 11
        previous = test_data_list[i]
        choice_list.insert(-1, previous['distractor1'])
        choice_list.insert(-1, previous['distractor2'])
    last_order = choice_list[-1]
    choice_list = choice_list[:-1]
#     print(current['answer'])
    shuffle(choice_list)
    choice_list.append(last_order)
    try:
        label = choice_list.index(current['answer'])
    except Exception as e:
        print(current['answer'])
        label = len(choice_list)-1
    new = {**current}
    new['label'] = label
    new['choice_list'] = choice_list
    return new

def test_with_k_answers(generated_data_list):
    """Testira radi li generiranje ispravno"""
    for original, new in zip(test_data_list, generated_data_list):
        assert set(original['choice_list']).issubset(set(new['choice_list'])), str(original) + '\n'+ str(new)

In [ ]:
random.seed(10)
generated_data_list = []
for i in range(len(test_data_list)):
#     new = create_with_k_answers(i, len(letters))
    new = create_with_ordered_correct_answer(i, 0)
#     new = create_with_question(i)
    generated_data_list.append(new)
# test_with_question(generated_data_list)
test_with_ordered_correct_answer(generated_data_list)
test_with_k_answers(generated_data_list)
generated_data_list[0]

## Testiranje

In [ ]:
generate(test_data_list[:10], tokens=1000, all_tokens=False, few_shot=False)

In [ ]:
for sample in test_data_list[:5]:
    print("\n>", sample['response'])

In [ ]:
generate(test_data_list, tokens=20, all_tokens=False, few_shot=False)

In [ ]:
word_play,sentence_play = getResultdata(test_data_list)
# word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play, sentence_play)

In [ ]:
save(name="flipped-letters", example=get_single_demo(test_data_list[0]))

In [ ]:
# save(few_shot=demonstration)
# save()